In [423]:
#sagemaker_raw_output = "{'source-ref': 'TBD', 'num-detected-objects': 1, 'bounding-box-attribute-name': {'image_size': [{'width': 640, 'height': 480, 'depth': 3}], 'annotations': [{'class_id': 0, 'left': 140, 'top': 129, 'width': 220, 'height': 286}]}, 'bounding-box-attribute-name-metadata': {'objects': [{'confidence': 0.63}], 'class-map': {'0': 'short sleeve top'}, 'type': 'descriptiveworld/object-detection', 'human-annotated': 'no', 'creation-date': '2021-11-16 01:47:10.657644', 'job-name': 'descriptive_world_identify_garments'}}"
sagemaker_raw_output = {'sageMakerOutput': 'eyJzb3VyY2UtcmVmIjogIlRCRCIsICJudW0tZGV0ZWN0ZWQtb2JqZWN0cyI6IDEsICJib3VuZGluZy1ib3gtYXR0cmlidXRlLW5hbWUiOiB7ImltYWdlX3NpemUiOiBbeyJ3aWR0aCI6IDY0MCwgImhlaWdodCI6IDQ4MCwgImRlcHRoIjogM31dLCAiYW5ub3RhdGlvbnMiOiBbeyJjbGFzc19pZCI6IDAsICJsZWZ0IjogMTQwLCAidG9wIjogMTI5LCAid2lkdGgiOiAyMjAsICJoZWlnaHQiOiAyODZ9XX0sICJib3VuZGluZy1ib3gtYXR0cmlidXRlLW5hbWUtbWV0YWRhdGEiOiB7Im9iamVjdHMiOiBbeyJjb25maWRlbmNlIjogMC42M31dLCAiY2xhc3MtbWFwIjogeyIwIjogInNob3J0IHNsZWV2ZSB0b3AifSwgInR5cGUiOiAiZGVzY3JpcHRpdmV3b3JsZC9vYmplY3QtZGV0ZWN0aW9uIiwgImh1bWFuLWFubm90YXRlZCI6ICJubyIsICJjcmVhdGlvbi1kYXRlIjogIjIwMjEtMTEtMTYgMDE6NDc6MTAuNjU3NjQ0IiwgImpvYi1uYW1lIjogImRlc2NyaXB0aXZlX3dvcmxkX2lkZW50aWZ5X2dhcm1lbnRzIn19', 'streamName': 'descriptiveworld-demo-kvs', 'frameMetaData': 'Frame(trackNumber=1, timeCode=0, keyFrame=true, invisible=false, discardable=false, lacing=NO)', 'fragmentMetaData': 'FragmentMetadata(fragmentNumberString=91343852333183046666493290777218913219824769829, serverSideTimestampMillis=1637008916339, producerSideTimestampMillis=1637008903613, fragmentNumber=91343852333183046666493290777218913219824769829, success=true, errorId=0, errorCode=null, millisBehindNow=OptionalLong[18312650], continuationToken=Optional[91343852333183046666493290777218913219824769829])'}

In [381]:
from __future__ import print_function
import base64
import json
import boto3
import os
from botocore.exceptions import ClientError

# set threshold for confidence
threshold = 0.75
            
def lambda_handler(event, context):
    for record in event['Records']:
        ###payload = base64.b64decode(record['kinesis']['data'])
        #Get Json format of Kinesis Data Stream Output
        ###result = json.loads(payload)
        result = sagemaker_raw_output
        # debug
        ###print("Raw Results:", result)
        #Get FragmentMetaData
        fragment = result['fragmentMetaData']
        print("fragment: " + fragment + "\n")
        #Get FrameMetaData
        frame = result['frameMetaData']
        print("frame: " + frame + "\n")
        #Get StreamName
        streamName = result['streamName']
        print("streamName: " + streamName + "\n")
        #Get SageMaker response and decode
        sageMakerOutput = json.loads(base64.b64decode(result['sageMakerOutput']))
        # Parse JSON into an object with attributes corresponding to dict keys.
        print("sagemaker raw output: " + str(sageMakerOutput))
        sageMakerOutput = json.loads('{"source-ref": "TBD","num-detected-objects": 3,"bounding-box-attribute-name":{"image_size": [{"width": 640, "height": 480, "depth": 3}],"annotations":[{"class_id": 0, "left": 140, "top": 129, "width": 220, "height": 286},{"class_id": 1, "left": 140, "top": 129, "width": 220, "height": 286},{"class_id": 1, "left": 140, "top": 129, "width": 220, "height": 286}]},"bounding-box-attribute-name-metadata":{"objects":[{"confidence": 0.63},{"confidence": 0.55},{"confidence": 0.75}],"class-map":{"0": "short sleeve top","1": "trousers"},"type": "descriptiveworld/object-detection","human-annotated": "no","creation-date": "2021-11-16 01:47:10.657644","job-name": "descriptive_world_identify_garments"}}')
        # First check to see if this is a new image, if not we don't do any inference
        # TODO
 
        # store the number of detected objects
        numObjects = int(sageMakerOutput['num-detected-objects'])
        
        # IF we have no objects, nothing left to do, let's end this lambda
        if (numObjects == 0):
            return None
        
        # although image size can appear with every object identification, we know it will always be the same
        imageSize = dict(sageMakerOutput['bounding-box-attribute-name']['image_size'][0])
        imageSizeHeight = imageSize['height']
        imageSizeWidth = imageSize['width']
        imageSizeDepth = imageSize['depth']
    
        # indices are not unique for foundObjects and boundingBoxConf
        # we will convert them to array of lists
        foundObjects = json.loads(json.dumps(sageMakerOutput['bounding-box-attribute-name']['annotations']))
        boundingBoxConfs = json.loads(json.dumps(sageMakerOutput['bounding-box-attribute-name-metadata']['objects']))
        
        # comes in as a list enclosed in {} let's convert to a dictionary since indices are unique
        boundingBoxClassMap = json.loads(json.dumps(sageMakerOutput['bounding-box-attribute-name-metadata']['class-map']))
    
        # debug
        print ("Number of Objects:", numObjects, "\nImage Size:", imageSize)
        print("Found Objects:", foundObjects, "\nBB Conf:", boundingBoxConfs, "\nBB Class Map:", boundingBoxClassMap, "\n")
        
        polly_sentence = "The following items are in front of you"
        # iterate over each one of the object found
        # This is the 'annotations', 'bounding-box-attribute-name-metadata'
        for i in range(0,numObjects):
            foundObject = foundObjects[i]
            boundingBoxConf = boundingBoxConfs[i]
            foundObjectClass_ID = foundObjects[i]['class_id']
            foundOjbectClassName = boundingBoxClassMap[str(foundObjectClass_ID)]
            print("Found Object:", foundObject, "Class Name:", foundOjbectClassName, "Confidence", boundingBoxConf, "\n")
            # append to polly sentence
            polly_sentence += ", " + foundOjbectClassName

        # Compose string for Amazon Polly
        print(polly_sentence)
        # Send string to amazon polly API to generate MP3
        
        # Draw bounding boxes on image
        # TODO 
        
        
    return 'Successfully processed {} records.'.format(len(event['Records']))

In [451]:
from __future__ import print_function
import base64
import json
import boto3
import os
from botocore.exceptions import ClientError
from datetime import datetime
import time
import calendar
import re

# set threshold for confidence
threshold = 0.75
# set region
region = "us-west-2"
stream_out_polly_name = "descriptive-demo-Post-Inference-Polly"
stream_out_polly_shards = 1
stream_out_imagebb_name = "descriptive-demo-Post-Inference-Image-Bounding-Box"
stream_out_imagebb_shards = 1
# setup kinesis client
kinesis_client = boto3.client('kinesis',region_name=region)

# method to write to the kinesis data stream
def put_to_stream(stream_out_name, data):
    timestamp = calendar.timegm(datetime.utcnow().timetuple())
    
    # JSON formatted payload
    payload = {
        'timestamp': int(timestamp),
        'data': data
    }
    print ("Payload for Kinesis DS: ", payload)
    put_response = kinesis_client.put_record(
        StreamName=stream_out_name,
        Data=json.dumps(payload),
        PartitionKey='inference')
        
# our  entry point            
def lambda_handler(event, context):
    for record in event['Records']:
        ###payload = base64.b64decode(record['kinesis']['data'])
        #Get Json format of Kinesis Data Stream Output
        ###result = json.loads(payload)
        result = sagemaker_raw_output

        print("START OF EXECUTION")
        # debug
        print("Raw Results:", result)
        #Get FragmentMetaData
        fragment = result['fragmentMetaData']
        print("fragment: " + fragment + "\n")
        #Get FrameMetaData
        frame = result['frameMetaData']
        print("frame: " + frame + "\n")
        #Get StreamName
        streamName = result['streamName']
        print("streamName: " + streamName + "\n")
        #Get SageMaker response and decode
        sageMakerOutput = json.loads(base64.b64decode(result['sageMakerOutput']))
        # Parse JSON into an object with attributes corresponding to dict keys.
        print("sagemaker raw output: " + str(sageMakerOutput))
        # First check to see if this is a new image, if not we don't do any inference
        # TODO
        
        # store the number of detected objects
        numObjects = int(sageMakerOutput['num-detected-objects'])
        
        # IF we have no objects, nothing left to do, let's end this lambda
        if (numObjects == 0):
            print("NO OBJECTS DETECTED, EXITING...")
            print("END OF EXECUTION")
            return None
        
        # record the source (should be full URI to image in S3)
        sourceRef = str(sageMakerOutput['source-ref'])
        
        # extract fragment number 
        fragmentNumber = re.search(r'^.*?\bfragmentNumber=(\d+),.*', fragment).group(1)
        
        # although image size can appear with every object identification, we know it will always be the same
        imageSize = dict(sageMakerOutput['bounding-box-attribute-name']['image_size'][0])
        imageSizeHeight = imageSize['height']
        imageSizeWidth = imageSize['width']
        imageSizeDepth = imageSize['depth']
    
        # indices are not unique for foundObjects and boundingBoxConf
        # we will convert them to array of lists
        foundObjects = json.loads(json.dumps(sageMakerOutput['bounding-box-attribute-name']['annotations']))
        boundingBoxConfs = json.loads(json.dumps(sageMakerOutput['bounding-box-attribute-name-metadata']['objects']))
        
        # comes in as a list enclosed in {} let's convert to a dictionary since indices are unique
        boundingBoxClassMap = json.loads(json.dumps(sageMakerOutput['bounding-box-attribute-name-metadata']['class-map']))
    
        # debug
        print ("Number of Objects:", numObjects, "\nImage Size:", imageSize)
        print("Found Objects:", foundObjects, "\nBB Conf:", boundingBoxConfs, "\nBB Class Map:", boundingBoxClassMap, "\n")
        
        polly_sentence = "There are " + str(numObjects) + " items in front of you"
        objects_detected = '{"class": "'
        # iterate over each one of the object found
        # This is the 'annotations', 'bounding-box-attribute-name-metadata'
        for i in range(0,numObjects):
            foundObject = foundObjects[i]
            boundingBoxConf = boundingBoxConfs[i]
            foundObjectClass_ID = foundObjects[i]['class_id']
            foundObjectClassName = boundingBoxClassMap[str(foundObjectClass_ID)]
            foundObjectLeft = foundObjects[i]['left']
            foundObjectTop = foundObjects[i]['top']
            foundObjectWidth = foundObjects[i]['width']
            foundObjectHeight = foundObjects[i]['height']
            print("Found Object:", foundObject, "Class Name:", foundObjectClassName, "Confidence", boundingBoxConf, "\n")
            # append to polly sentence
            polly_sentence += ", " + foundObjectClassName
            # append to objects JSON string
            objects_detected += str(foundObjectClassName) + '", "confidence": ' + str(boundingBoxConf['confidence']) + ', "left": ' + str(foundObjectLeft) + ', "top": ' +  \
                str(foundObjectTop) + ', "width": ' + str(foundObjectWidth) + ', "height": ' + str(foundObjectHeight)
            if (i < numObjects-1):
                objects_detected += ', ' 

        objects_detected += '}'
        polly_sentence += "."
        # reformat objects_detected as JSON
        objects_detected = json.loads(objects_detected)
        
        # Print complete sentence for Amazon Polly
        print(polly_sentence)

        
        # construct JSON for data payload for polly
        data_polly = {
            'fragment-number': int(fragmentNumber),
            'sentence' : polly_sentence
        }
        
        # Now send all data to the kinesis data stream (to be picked up for polly narration)
        put_to_stream(stream_out_polly_name, data_polly)
        
        # construct JSON for data payload for polly
        data_imagebb = {
            'source-ref': sourceRef,
            'fragment-number': int(fragmentNumber),
            'num-detected-objects': numObjects,
            'image-size': imageSize,
            'objects': objects_detected
        }
        # Now send all data to the kinesis data stream (to be picked up for polly narration)
        put_to_stream(stream_out_imagebb_name, data_imagebb)
        
        print(str(numObjects) + " OBJECTS DETECTED")
        print("END OF EXECUTION")
        
    return 'Successfully processed {} records.'.format(len(event['Records']))


In [452]:
sm_event = dict()
sm_event['Records'] = ['kinesis']
records_processed = (lambda_handler(sm_event, ""))
print(records_processed)

START OF EXECUTION
Raw Results: {'sageMakerOutput': 'eyJzb3VyY2UtcmVmIjogIlRCRCIsICJudW0tZGV0ZWN0ZWQtb2JqZWN0cyI6IDEsICJib3VuZGluZy1ib3gtYXR0cmlidXRlLW5hbWUiOiB7ImltYWdlX3NpemUiOiBbeyJ3aWR0aCI6IDY0MCwgImhlaWdodCI6IDQ4MCwgImRlcHRoIjogM31dLCAiYW5ub3RhdGlvbnMiOiBbeyJjbGFzc19pZCI6IDAsICJsZWZ0IjogMTQwLCAidG9wIjogMTI5LCAid2lkdGgiOiAyMjAsICJoZWlnaHQiOiAyODZ9XX0sICJib3VuZGluZy1ib3gtYXR0cmlidXRlLW5hbWUtbWV0YWRhdGEiOiB7Im9iamVjdHMiOiBbeyJjb25maWRlbmNlIjogMC42M31dLCAiY2xhc3MtbWFwIjogeyIwIjogInNob3J0IHNsZWV2ZSB0b3AifSwgInR5cGUiOiAiZGVzY3JpcHRpdmV3b3JsZC9vYmplY3QtZGV0ZWN0aW9uIiwgImh1bWFuLWFubm90YXRlZCI6ICJubyIsICJjcmVhdGlvbi1kYXRlIjogIjIwMjEtMTEtMTYgMDE6NDc6MTAuNjU3NjQ0IiwgImpvYi1uYW1lIjogImRlc2NyaXB0aXZlX3dvcmxkX2lkZW50aWZ5X2dhcm1lbnRzIn19', 'streamName': 'descriptiveworld-demo-kvs', 'frameMetaData': 'Frame(trackNumber=1, timeCode=0, keyFrame=true, invisible=false, discardable=false, lacing=NO)', 'fragmentMetaData': 'FragmentMetadata(fragmentNumberString=9134385233318304666649329077721891321982476